<a href="https://colab.research.google.com/github/ARU-Bioinformatics/advanced-programming-SBV/blob/main/Differential_expression_analysis_in_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Differential gene expression analysis using R

### Prepare the working environment 
This first code cell installs the needed packages (leave 30 mins for this to be run)

*   Install the subread package
*   Install the seqinr package
*   Install the DESeq2 package

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
install.packages("ggpubr")
BiocManager::install("Rsubread")
BiocManager::install("DESeq2")
install.packages("seqinr")

Load the required packages for the first step of the process

In [ ]:
library(Rsubread)
library(seqinr)

## Part 1.
The following steps go through how to do get count data from fastq files. In a previous week we did this from the command line, and used kallisto quant. You could input your data from that notebook directly into section 2 below. Part 1 is included here to show an additional way of generating RNA-seq count, and you will see how thgis format is directly ammeanable to DESeq2.

In [ ]:
# Create and set the working directory
dir.create("~/Demo_alignment")
setwd("~/Demo_alignment")

### From the raw reads to the aligned reads: single end
*   In this step we upload and inspect the reference fasta file and the reads 
*   We are using demo reference reads fasta files; the lines below assign to the variable ''reference'' and ''reads'' the path to the fasta file  and reads file respectively

In [ ]:
reference <- system.file("extdata","reference.fa",package="Rsubread")
reads <- system.file("extdata","reads.txt.gz",package="Rsubread")

# Saves the reads and fasta files so that we can  inspect it  

reference_sample <- read.delim(reference, header = F)
reads_sample <- read.delim(reads, header = F)

write.table(reference_sample, file = "reference.fa", row.names = F, col.names = F, quote = F) 
write.table(reads_sample, file = "reads.fq", row.names = F, col.names = F, quote = F) 

*    Similar to using commandline tools we can perform some initial stats on the reads. We will express in numbers the phread quality score of each read

In [ ]:
qual_score <- qualityScores(filename=reads,offset=64,nreads=1000)

View(qual_score)

#plots a sample quality score
plot(qual_score[1,], type="h")

atgcContent(reads)

*    Now we can perform and process the alignment


In [ ]:
# Builds an index based on the reference fasta file
buildindex(basename="./reference_index",reference=reference)

# Aligns the reads to the reference
align.stat_SE <- align(index = "./reference_index", readfile1 = reads, output_format = "SAM",
                    output_file = "./aligned_reads_SE.SAM", phredOffset = 64, nthreads=5)

# Prints the alignment statistics on the console
align.stat_SE

### From the raw reads to the aligned reads: paired end
*   The following steps show you how to process paired end data (which ideally in 2022 would be what we handled)

In [ ]:
reads1 <- system.file("extdata","reads1.txt.gz",package="Rsubread")
reads2 <- system.file("extdata","reads2.txt.gz",package="Rsubread")

In [ ]:
# Saves the reads files so that we can  inspect it  

reads_sample1 <- read.delim(reads1, header = F)
reads_sample2 <- read.delim(reads2, header = F)

write.table(reads_sample1, file = "reads1.fq", row.names = F, col.names = F, quote = F) 
write.table(reads_sample2, file = "reads2.fq", row.names = F, col.names = F, quote = F) 

# Aligns the reads to the reference
align.stat_PE <- align(index="reference_index",readfile1=reads1,readfile2=reads2,output_format = "SAM", output_file="aligned_reads_PE.SAM", phredOffset=64, nthreads=5)

# Prints the alignment statistics on the console
align.stat_PE

### From aligned reads to counts
*   Now that we have aligned reads we can generate counts. 
*   Our first step is having the annotations of genes in the reference genome. Here we are writing the file for ourselves, but you can download the annotation in bed format for any trasncriptome for this stage.


In [ ]:
# Annotates the reference fasta file
Annotation <- data.frame(GeneID=c("gene1","gene1","gene2","gene2"), 
                         Chr="chr_dummy", Start=c(100,1000,3000,5000),
                         End=c(500,1800,4000,5500), 
                         Strand=c("+","+","-","-"), 
                         stringsAsFactors=FALSE)

# Prints the Annotation dataframe on the console
Annotation

In [ ]:
# Uses featureCounts to assign the reads to the annotated features

Counts_SE <- featureCounts("aligned_reads_SE.SAM",annot.ext=Annotation)

Counts_PE <- featureCounts("aligned_reads_PE.SAM",annot.ext=Annotation, isPairedEnd=TRUE)

# Prints the counts per gene on the console
Counts_SE$counts

#Prints the counts statistics on the console

Counts_SE$stat

print("So far, so good, part 1 completed!")


## Part 2 - identifiying differentially expressed genes
*   Our first step is to load the required package - DESeq2
*   These steps take an alignment performed using the packages from part 1. If you are using the input from kallisto quant then you will need to explore your matrix file and get it to resemble the output above.


In [ ]:
library("DESeq2")

### Activity 1:
For DESe12 to be run we need a table summarising the samples used in the experiment. The table layout and name for this experiment is given below. Write code in R to make this table as a data frame and save as an object called `coldata`

In [ ]:
##            condition        type
## reads1     treated paired-read
## reads2     untreated  paired-end

In [ ]:
# my code to write a data frame called coldata that looks like the table above

*    The output from Rsubread can be inputted straight into DESeq2, using the `DESeqDataSetFromMatrix` function.

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = Counts_PE,
                              colData = coldata,
                              design = ~ condition)
dds

*   Now we have inpout ready for doing our statisitical tests of differnetial expression.

In [ ]:
dds <- DESeq(dds)
res <- results(dds)
res

*    Plot the results using the inbuilt function to use base R plots

In [ ]:
plotMA(res, ylim=c(-2,2))

To make quick nice DE plots use ggpubr. As the name hints, this package makes publication ready plots using ggplot. It has a nice preloaded dataset called diff_express that can be used to showcase what it can do! The code cells below give you some examples with that dataset.

In [ ]:
# First have a look at the data, so you know what you are plotting
head(diff_express)

In [ ]:
# Default plot
ggmaplot(diff_express, main = expression("Group 1" %->% "Group 2"),
   fdr = 0.05, fc = 2, size = 0.4,
   palette = c("#B31B21", "#1465AC", "darkgray"),
   genenames = as.vector(diff_express$name),
   legend = "top", top = 20,
   font.label = c("bold", 11),
   font.legend = "bold",
   font.main = "bold",
   ggtheme = ggplot2::theme_minimal())

In [ ]:
# Add rectangle around labesl
ggmaplot(diff_express, main = expression("Group 1" %->% "Group 2"),
   fdr = 0.05, fc = 2, size = 0.4,
   palette = c("#B31B21", "#1465AC", "darkgray"),
   genenames = as.vector(diff_express$name),
   legend = "top", top = 20,
   font.label = c("bold", 11), label.rectangle = TRUE,
   font.legend = "bold",
   font.main = "bold",
   ggtheme = ggplot2::theme_minimal())

### Activity 2: 
Now you know how to do differential expression run the notebook, and then add in some code below to make a nice ggpubr graph of the differential expression data you analysed above.

In [ ]:
## your code for ggpubr here